In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['patch.force_edgecolor'] = True
import seaborn as sns

from sklearn import model_selection, metrics
from sklearn import linear_model, feature_extraction
from sklearn import pipeline

Загрузите датасет.

In [30]:
df = pd.read_csv('data/SMSSpamCollection.txt',
                 delimiter='\t', header=None)
df.columns = ['spam', 'text']
df.head()

,spam,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [31]:
df['spam'] = df['spam'].map({'ham': 0, 'spam': 1})
df.head()

,spam,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


___
Получите из списка текстов матрицу признаков X.

In [23]:
f_extr = feature_extraction.text.CountVectorizer()

In [24]:
text_feat_matrix = f_extr.fit_transform(df.text)

___
Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [36]:
lr_clf = linear_model.LogisticRegression(random_state=2)

In [126]:
%%time
answer = model_selection.cross_val_score(
    lr_clf,
    text_feat_matrix,
    df.spam,
    cv=10, n_jobs=-1, scoring='f1').mean()
print('Score:', answer)

0.9326402983610631
CPU times: user 48.4 ms, sys: 80.4 ms, total: 129 ms
Wall time: 1.39 s


In [56]:
with open('w3_ans1', 'w') as fout:
    fout.write(str(f'{answer:.1f}'))

___
Обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

    "FreeMsg: Txt: claim your reward of 3 hours talk time"

    "Have you visited the last lecture on physics?"

    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

    "Only 99$"

In [61]:
lr_clf.fit(f_matrix, df.spam)

/Users/artem/miniconda3/envs/mldaspec/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=2, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [62]:
test_msgs = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"
]

In [72]:
clfr_pipe = pipeline.Pipeline(
    [('vecorizer', feature_extraction.text.CountVectorizer()),
     ('classifier', linear_model.LogisticRegression(random_state=2))]
)

In [73]:
%%time
clfr_pipe.fit(df.text, df.spam)

CPU times: user 173 ms, sys: 6.64 ms, total: 180 ms
Wall time: 114 ms


/Users/artem/miniconda3/envs/mldaspec/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vecorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...e, penalty='l2', random_state=2, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [75]:
test_msgs[0]

'FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB'

In [105]:
convert = {0: 'Ham', 1: 'Spam'}
answer2 = []
for msg in test_msgs:
    print(msg)
    spam = clfr_pipe.predict([msg])[0]
    print('Class:', spam)
    answer2.append(spam)

FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB
Class: 1
FreeMsg: Txt: claim your reward of 3 hours talk time
Class: 1
Have you visited the last lecture on physics?
Class: 0
Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$
Class: 0
Only 99$
Class: 0


In [106]:
answer2

[1, 1, 0, 0, 0]

In [124]:
with open('w3_ans2', 'w') as fout:
    fout.write(' '.join([str(x) for x in answer2]))

___
Задайте в `CountVectorizer` параметр `ngram_range=(2,2)`, затем `ngram_range=(3,3)`, затем `ngram_range=(1,3)`. 

Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [134]:
score_arr = []
for ngram_r in [(2, 2), (3, 3), (1, 3)]:
    print(ngram_r)
    score = model_selection.cross_val_score(
        pipeline.Pipeline(
            [('vectorizer', feature_extraction.text.CountVectorizer(ngram_range=ngram_r)),
             ('classifier', linear_model.LogisticRegression(random_state=2))]
        ),
        df.text,
        df.spam,
        cv=10,
        scoring='f1',
        n_jobs=-1
    ).mean()
    score_arr.append(score)
print(score_arr)

(2, 2)
(3, 3)
(1, 3)
[0.8224220664187133, 0.7250161555467377, 0.9251382558648837]


In [136]:
with open('w3_ans3', 'w') as fout:
    fout.write(' '.join([str(round(x, 2)) for x in score_arr]))
    

In [137]:
!cat w3_ans3

0.82 0.73 0.93

___
Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии `MultinomialNB()`. Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

*По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.*

In [138]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_extraction.text import CountVectorizer

In [149]:
bigrams = CountVectorizer(ngram_range=(2,2)).fit_transform(df.text)
trigrams = CountVectorizer(ngram_range=(3,3)).fit_transform(df.text)
uni_bi_trigrams = CountVectorizer(ngram_range=(1,3)).fit_transform(df.text)

In [155]:
score_arr = []
for features in [bigrams, trigrams, uni_bi_trigrams]:
    score = model_selection.cross_val_score(
        MultinomialNB(),
        features,
        df.spam,
        cv=10,
        scoring='f1',
        n_jobs=-1
    ).mean()
    score_arr.append(score)

print(score_arr)

[0.6455015177985443, 0.37871948524573595, 0.8884859656061002]


In [156]:
with open('w3_ans4', 'w') as fout:
    fout.write(' '.join([str(round(x, 2)) for x in score_arr]))

In [157]:
!cat w3_ans4

0.65 0.38 0.89

___
Попробуйте использовать в логистической регрессии в качестве признаков `Tf*idf` из `TfidfVectorizer` на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с `CountVectorizer` на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к `tf*idf` не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.


In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [160]:
tfidf_features = TfidfVectorizer().fit_transform(df.text)

In [161]:
model_selection.cross_val_score(
    linear_model.LogisticRegression(),
    tfidf_features,
    df.spam,
    cv=10,
    scoring='f1',
    n_jobs=-1
).mean()

0.8528599554172456

In [162]:
with open('w3_ans5.txt', 'w') as fout:
    fout.write('-1')
!cat w3_ans5

-1